In [125]:
# %reset

In [126]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

%matplotlib inline

In [ ]:
data = pd.read_csv('kaggle/Multi-Class Prediction of Obesity Risk/train.csv', index_col='id')
data

In [ ]:
data['NObeyesdad'].value_counts()

In [129]:
# разделим сразу

y = data['NObeyesdad']
data = data.drop('NObeyesdad', axis=1)

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
cat_features = data.select_dtypes(include=['object', 'category']).columns.tolist()
cat_features

In [136]:
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# Трансформация категориальных признаков
encoded_cat_data = ohe.fit_transform(data[cat_features])

# Создание DataFrame с закодированными признаками
encoded_cat_data_df = pd.DataFrame(encoded_cat_data, columns=ohe.get_feature_names_out(cat_features))

# Объединение закодированных категориальных признаков с исходным датасетом
data = pd.concat([data.drop(columns=cat_features), encoded_cat_data_df], axis=1)

Обработка выбросов: Некоторые числовые признаки, такие как Weight (вес) и Height (рост), могут содержать выбросы. Можно проверить наличие выбросов с помощью графиков (например, ящиков с усами) и решить, стоит ли их удалять или заменять.

In [ ]:
# смотрим выбросы

import matplotlib.pyplot as plt
import seaborn as sns

# Установка стиля графиков
sns.set(style="whitegrid")

# Создание графика "ящик с усами" для признака Weight
plt.figure(figsize=(8, 6))
sns.boxplot(data=data, x='Weight')
plt.title('Boxplot of Weight')
plt.show()

# Создание графика "ящик с усами" для признака Height
plt.figure(figsize=(8, 6))
sns.boxplot(data=data, x='Height')
plt.title('Boxplot of Height')
plt.show()

In [137]:
# обработаем, закодируем таргет

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

После обучения модели и получения предсказаний преобразуем их обратно в исходные метки
predicted_labels = label_encoder.inverse_transform(predicted_classes)

In [140]:
# создаем новый столбец отношение массы к росту

data['BMI'] = data['Weight'] / (data['Height'] ** 2)

In [141]:
# копируем в X - просто так

# Теперь X включает новый признак BMI
X = data.copy()

In [ ]:
# найдем все числовые столбцы

num_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_features

In [143]:
# масштабирование
scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])

# ПОДБОР и ОБУЧЕНИЕ МОДЕЛИ

In [144]:
# Инициализация моделей
logistic_regression = LogisticRegression()
random_forest = RandomForestClassifier()
gradient_boosting = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

### Подбор параметров для Логистической регрессии

In [145]:
# Подбор параметров для Логистической регрессии
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100]  # C и 5 чисел следом - это подбор коэф. регуляризации
}

# GridSearchCV для логистической регрессии
grid_search_lr = GridSearchCV(logistic_regression, param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X, y_encoded)

# Лучшие параметры и оценка для логистической регрессии
print("Лучшие параметры (логистическая регрессия):", grid_search_lr.best_params_)
print("Лучшая оценка (логистическая регрессия):", grid_search_lr.best_score_)

Лучшие параметры (логистическая регрессия): {'C': 10}
Лучшая оценка (логистическая регрессия): 0.8665097065544483


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Подбор параметров для Случайного леса

In [146]:
# Параметры для случайного леса
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV для случайного леса
grid_search_rf = GridSearchCV(random_forest, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X, y_encoded)

# Лучшие параметры и оценка для случайного леса
print("Лучшие параметры (случайный лес):", grid_search_rf.best_params_)
print("Лучшая оценка (случайный лес):", grid_search_rf.best_score_)

Лучшие параметры (случайный лес): {'max_depth': 30, 'min_samples_split': 5, 'n_estimators': 200}
Лучшая оценка (случайный лес): 0.90182132215976


### Подбор параметров для градиентного бустинга

In [147]:
# Параметры для градиентного бустинга
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# GridSearchCV для градиентного бустинга
grid_search_gb = GridSearchCV(gradient_boosting, param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X, y_encoded)

# Лучшие параметры и оценка для градиентного бустинга
print("Лучшие параметры (градиентный бустинг):", grid_search_gb.best_params_)
print("Лучшая оценка (градиентный бустинг):", grid_search_gb.best_score_)

Лучшие параметры (градиентный бустинг): {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Лучшая оценка (градиентный бустинг): 0.9077463981332817


### Обучение моделей на тренировочных данных

In [148]:
# Обучение моделей с лучшими параметрами
logistic_regression_best = LogisticRegression(C=10)
random_forest_best = RandomForestClassifier(max_depth=20, min_samples_split=2, n_estimators=200)
gradient_boosting_best = GradientBoostingClassifier(learning_rate=0.2, max_depth=3, n_estimators=200)

logistic_regression_best.fit(X, y_encoded)
random_forest_best.fit(X, y_encoded)
gradient_boosting_best.fit(X, y_encoded)

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GradientBoostingClassifier(learning_rate=0.2, n_estimators=200)

### Загрузка и предобработка тестового датасета

In [166]:
# Загрузка тестового датасета
test_data = pd.read_csv('kaggle/Multi-Class Prediction of Obesity Risk/test.csv', index_col='id')

In [167]:
from sklearn.impute import SimpleImputer

# Предобработка тестового датасета (аналогично тренировочному)
encoded_cat_test_data = ohe.transform(test_data[cat_features])
encoded_cat_test_data_df = pd.DataFrame(encoded_cat_test_data, columns=ohe.get_feature_names_out(cat_features), index=test_data.index)
test_data = pd.concat([test_data.drop(columns=cat_features), encoded_cat_test_data_df], axis=1)
test_data['BMI'] = test_data['Weight'] / (test_data['Height'] ** 2)

# Импьютация для заполнения пропущенных значений
imputer = SimpleImputer(strategy='mean')
test_data[num_features] = imputer.fit_transform(test_data[num_features])

# Масштабирование числовых признаков
test_data[num_features] = scaler.transform(test_data[num_features])

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [168]:
# Предсказания моделей
predictions_lr = logistic_regression_best.predict(test_data)
predictions_rf = random_forest_best.predict(test_data)
predictions_gb = gradient_boosting_best.predict(test_data)

# Преобразование предсказаний обратно в исходные метки
predicted_labels_lr = label_encoder.inverse_transform(predictions_lr)
predicted_labels_rf = label_encoder.inverse_transform(predictions_rf)
predicted_labels_gb = label_encoder.inverse_transform(predictions_gb)

In [169]:
# Сохранение предсказаний логистической регрессии
submission_lr = pd.DataFrame({'id': test_data.index, 'NObeyesdad': predicted_labels_lr})
submission_lr.to_csv('submission_lr.csv', index=False)

# Сохранение предсказаний случайного леса
submission_rf = pd.DataFrame({'id': test_data.index, 'NObeyesdad': predicted_labels_rf})
submission_rf.to_csv('submission_rf.csv', index=False)

# Сохранение предсказаний градиентного бустинга
submission_gb = pd.DataFrame({'id': test_data.index, 'NObeyesdad': predicted_labels_gb})
submission_gb.to_csv('submission_gb.csv', index=False)

ПЛАН:

Загрузка тренировочного датасета.
Проверка данных на наличие пропущенных значений и дубликатов.
Кодирование категориальных признаков.
Разделение датасета на признаки (X) и целевую переменную (y).
Масштабирование числовых признаков.

Выбор моделей для обучения (в данном случае логистическая регрессия, случайный лес и градиентный бустинг).
Подбор параметров для моделей с использованием кросс-валидации и GridSearchCV (в данном случае выполнено только для логистической регрессии).
Обучение моделей на тренировочных данных.
Загрузка тестового датасета и проверка на наличие категорий, которых нет в тренировочном датасете. Заменить все NaN.
Предобработка тестового датасета (аналогично тренировочному датасету).
Сделать предсказания на тестовых данных с использованием обученных моделей.
Сохранение предсказаний в формате CSV для отправки в Kaggle.

Привет, я начинающий специалист по МЛ. Собираюсь справится с тренировочным соревнованием на Kaggle. Вот описание на англ. ниже. А также тренировочный датасет я прикрепил изучи его. на нем обучим, его делить на тестовую выборку ненадо тк у меня есть отдельно тестовый датасет test.csv

Пожалуйста думай внимательно шаг за шагом, будь как хороший специалист который не упускает важные детали. с начала нужно обработать дата сет, убрать пустые значения, найти повторы и тп.

Дальше мне необходимо выбрать 3 модели логистическая регрессия, случайный лес и градиентный бустинг и помоги мне используя кросс валидацию, метод pipeline перебрать параметры grid_params, также используя gridsearch - найти лучшие параметры и на них сделать предсказание X_test в конце выгрузить результат в виде csv файла sample_submission:



Submission File
For each id row in the test set, you must predict the class value of the target, NObeyesdad. The file should contain a header and have the following format:

id,NObeyesdad
20758,Normal_Weight
20759,Normal_Weight
20760,Normal_Weight
etc.



вот описание соревнования:

Overview
Welcome to the 2024 Kaggle Playground Series! We plan to continue in the spirit of previous playgrounds, providing interesting an approachable datasets for our community to practice their machine learning skills, and anticipate a competition each month.

Your Goal: The goal of this competition is to use various factors to predict obesity risk in individuals, which is related to cardiovascular disease. Good luck!

Evaluation
Submissions are evaluated using the accuracy score.

About the Tabular Playground Series
The goal of the Tabular Playground Series is to provide the Kaggle community with a variety of fairly light-weight challenges that can be used to learn and sharpen skills in different aspects of machine learning and data science. The duration of each competition will generally only last a few weeks, and may have longer or shorter durations depending on the challenge. The challenges will generally use fairly light-weight datasets that are synthetically generated from real-world data, and will provide an opportunity to quickly iterate through various model and feature engineering ideas, create visualizations, etc.

Synthetically-Generated Datasets
Using synthetic data for Playground competitions allows us to strike a balance between having real-world data (with named features) and ensuring test labels are not publicly available. This allows us to host competitions with more interesting datasets than in the past. While there are still challenges with synthetic data generation, the state-of-the-art is much better now than when we started the Tabular Playground Series two years ago, and that goal is to produce datasets that have far fewer artifacts. Please feel free to give us feedback on the datasets for the different competitions so that we can continue to improve!



вот мой план:

Загрузка тренировочного датасета.
Проверка данных на наличие пропущенных значений и дубликатов.
Кодирование категориальных признаков.
Разделение датасета на признаки (X) и целевую переменную (y).
Масштабирование числовых признаков.
Выбор моделей для обучения (в данном случае логистическая регрессия, случайный лес и градиентный бустинг).
Подбор параметров для моделей с использованием кросс-валидации и GridSearchCV (в данном случае выполнено только для логистической регрессии).
Обучение моделей на тренировочных данных.
Загрузка тестового датасета и проверка на наличие категорий, которых нет в тренировочном датасете. Заменить все NaN.
Предобработка тестового датасета (аналогично тренировочному датасету).
Сделать предсказания на тестовых данных с использованием обученных моделей.
Сохранение предсказаний в формате CSV для отправки в Kaggle.

подумай хорошо, не торопись, вдохни полную грудь и поехали по шагам, внимательно, я дам тебе чаевые 20$

Вот несколько способов, как ты можешь проверить тестовый датасет на наличие категорий, которых нет в тренировочном датасете:

1. Сравнение уникальных значений в категориальных признаках
Для каждого категориального признака ты можешь сравнить уникальные значения в тренировочном и тестовом датасетах:

In [ ]:
for feature in cat_features:
    train_unique = set(data[feature].unique())
    test_unique = set(test_data[feature].unique())
    
    # Находим значения, которые есть в тестовом, но нет в тренировочном датасете
    diff = test_unique - train_unique
    if diff:
        print(f"В признаке '{feature}' следующие категории есть в тестовом датасете, но отсутствуют в тренировочном: {diff}")
    else:
        print(f"В признаке '{feature}' нет расхождений в категориях между тренировочным и тестовым датасетами.")

2. Использование OneHotEncoder с параметром handle_unknown='error'
Ты можешь использовать OneHotEncoder с параметром handle_unknown='error', чтобы вызвать ошибку, если в тестовом датасете обнаружатся неизвестные категории:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='error')
ohe.fit(data[cat_features])

try:
    ohe.transform(test_data[cat_features])
    print("В тестовом датасете не обнаружено неизвестных категорий.")
except ValueError as e:
    print("Обнаружены неизвестные категории в тестовом датасете:", e)

3. Использование pandas.get_dummies с параметром dummy_na=True
Если ты используешь pandas.get_dummies для кодирования категориальных признаков, ты можешь установить параметр dummy_na=True, чтобы создать дополнительный столбец для NaN значений. Если в тестовом датасете есть категории, которых нет в тренировочном датасете, они будут закодированы как NaN:

In [ ]:
train_dummies = pd.get_dummies(data[cat_features], drop_first=True, dummy_na=True)
test_dummies = pd.get_dummies(test_data[cat_features], drop_first=True, dummy_na=True)

# Проверка на наличие столбцов с NaN в тестовом датасете
nan_columns = [col for col in test_dummies.columns if col.endswith('_nan')]
if nan_columns:
    print("В тестовом датасете обнаружены неизвестные категории, закодированные как NaN:", nan_columns)
else:
    print("В тестовом датасете не обнаружено неизвестных категорий.")